In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 경고 뜨지 않게
import warnings
warnings.filterwarnings('ignore')

#저장라이브러리
import pickle

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# LabelEncoder
from sklearn.preprocessing import LabelEncoder

# 데이터를 학습용과 검증용으로 나눈다.
from sklearn.model_selection import train_test_split

import os

In [2]:
# 데이터 불러오기
df1 = pd.read_csv('EDA/unique제거/train_3_매출정보_unique.csv')
df2 = pd.read_csv('EDA/Segment/Segment_병합.csv')

In [3]:
# 열 기준으로 단순 병합
merged_df = pd.concat([df1.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)

print("✅ 열 기준으로 concat 완료")
print(merged_df.shape)
display(merged_df.head())

✅ 열 기준으로 concat 완료
(2400000, 383)


,기준년월,ID,최종이용일자_기본,최종이용일자_신판,최종이용일자_CA,최종이용일자_카드론,최종이용일자_체크,최종이용일자_일시불,최종이용일자_할부,이용건수_신용_B0M,...,승인거절건수_BL_B0M,승인거절건수_기타_B0M,승인거절건수_R3M,승인거절건수_한도초과_R3M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,승인거절건수_기타_R3M,이용금액대,ID,Segment
0,201807,TRAIN_000000,20180719,20180713,20180719,미이용,20180203,20180709,20180713,11,...,0,0,3,3,0,0,0,01.100만원+,TRAIN_000000,D
1,201807,TRAIN_000001,20180719,20180719,20170728,20170327,미이용,20180719,20171231,13,...,0,0,3,3,0,0,0,03.30만원+,TRAIN_000001,E
2,201807,TRAIN_000002,20180706,20180706,20180706,20151119,20141230,20180706,20180627,12,...,0,0,0,0,0,0,0,01.100만원+,TRAIN_000002,C
3,201807,TRAIN_000003,20180721,20180715,20180721,미이용,20141111,20180704,20180715,6,...,0,0,3,3,0,0,0,01.100만원+,TRAIN_000003,D
4,201807,TRAIN_000004,20180124,20180124,미이용,미이용,20180512,20180124,미이용,-2,...,0,0,0,0,0,0,0,09.미사용,TRAIN_000004,E


In [4]:
merged_df['Segment']

0          D
1          E
2          C
3          D
4          E
          ..
2399995    E
2399996    D
2399997    C
2399998    E
2399999    E
Name: Segment, Length: 2400000, dtype: object

In [5]:
# 최빈값 98% 이상 제거한 컬럼 확인
cols_to_drop = ['이용건수_카드론_B0M', 
                '이용금액_카드론_B0M', 
                '이용후경과월_부분무이자',
                '이용금액_부분무이자_R12M',
                '이용건수_부분무이자_R6M',
                '이용금액_카드론_R6M',
                '이용개월수_부분무이자_R6M',
                '이용건수_부분무이자_R3M',
                '이용건수_카드론_R3M',
                '이용개월수_부분무이자_R3M',
                '이용개월수_카드론_R3M',
                '교통_통행료이용금액',
                '여유_Pet이용금액',
                '여유_공원이용금액',
                '여유_숙박이용금액',
                '여유_항공이용금액',
                '_3순위여유업종',
                '_3순위여유업종_이용금액',
                '할부건수_12M_R12M',
                '할부건수_14M_R12M',
                '할부건수_유이자_12M_R12M',
                '할부금액_유이자_12M_R12M',
                '할부건수_무이자_14M_R12M',
                '할부금액_무이자_14M_R12M',
                '할부금액_부분_6M_R12M',
                '할부금액_부분_14M_R12M',
                'RP건수_렌탈_B0M',
                'RP건수_가스_B0M',
                'RP건수_전기_B0M',
                'RP건수_보험_B0M',
                'RP건수_학습비_B0M',
                'RP후경과월_렌탈',
                'RP후경과월_가스',
                'RP후경과월_전기',
                'RP후경과월_보험',
                'RP후경과월_학습비',
                '증감_RP건수_통신_전월',
                '증감_RP건수_렌탈_전월',
                '증감_RP건수_학습비_전월',
                '이용금액_D페이_R3M',
                '이용건수_B페이_R3M',
                '이용건수_D페이_R3M',
                '이용금액_B페이_B0M',
                '이용금액_D페이_B0M',
                '이용건수_D페이_B0M',
                '이용건수_선결제_B0M',
                '건수_할부전환_R6M',
                '건수_할부전환_R3M',
                '금액_할부전환_R6M',
                '금액_할부전환_R3M',
                '신청건수_ATM_CA_B0',
                '신청건수_ATM_CL_B0',
                '승인거절건수_B0M',
                '승인거절건수_한도초과_B0M',
                '승인거절건수_BL_B0M',
                '승인거절건수_BL_R3M',
                '승인거절건수_입력오류_R3M',
                'Segment'
               ]

In [6]:
selected_df = merged_df[cols_to_drop].copy()

print("✅ 선택한 컬럼들만 남긴 데이터프레임 샘플:")
display(selected_df.head())

✅ 선택한 컬럼들만 남긴 데이터프레임 샘플:


,이용건수_카드론_B0M,이용금액_카드론_B0M,이용후경과월_부분무이자,이용금액_부분무이자_R12M,이용건수_부분무이자_R6M,이용금액_카드론_R6M,이용개월수_부분무이자_R6M,이용건수_부분무이자_R3M,이용건수_카드론_R3M,이용개월수_부분무이자_R3M,...,금액_할부전환_R6M,금액_할부전환_R3M,신청건수_ATM_CA_B0,신청건수_ATM_CL_B0,승인거절건수_B0M,승인거절건수_한도초과_B0M,승인거절건수_BL_B0M,승인거절건수_BL_R3M,승인거절건수_입력오류_R3M,Segment
0,0,0,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,D
1,0,0,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
2,0,0,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,C
3,0,0,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,D
4,0,0,12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E


In [7]:
selected_df['Segment'].value_counts()

Segment
E    1922052
D     349242
C     127590
A        972
B        144
Name: count, dtype: int64

In [8]:
print("✅ 선택된 컬럼명 목록:")
print(selected_df.columns.tolist())

✅ 선택된 컬럼명 목록:
['이용건수_카드론_B0M', '이용금액_카드론_B0M', '이용후경과월_부분무이자', '이용금액_부분무이자_R12M', '이용건수_부분무이자_R6M', '이용금액_카드론_R6M', '이용개월수_부분무이자_R6M', '이용건수_부분무이자_R3M', '이용건수_카드론_R3M', '이용개월수_부분무이자_R3M', '이용개월수_카드론_R3M', '교통_통행료이용금액', '여유_Pet이용금액', '여유_공원이용금액', '여유_숙박이용금액', '여유_항공이용금액', '_3순위여유업종', '_3순위여유업종_이용금액', '할부건수_12M_R12M', '할부건수_14M_R12M', '할부건수_유이자_12M_R12M', '할부금액_유이자_12M_R12M', '할부건수_무이자_14M_R12M', '할부금액_무이자_14M_R12M', '할부금액_부분_6M_R12M', '할부금액_부분_14M_R12M', 'RP건수_렌탈_B0M', 'RP건수_가스_B0M', 'RP건수_전기_B0M', 'RP건수_보험_B0M', 'RP건수_학습비_B0M', 'RP후경과월_렌탈', 'RP후경과월_가스', 'RP후경과월_전기', 'RP후경과월_보험', 'RP후경과월_학습비', '증감_RP건수_통신_전월', '증감_RP건수_렌탈_전월', '증감_RP건수_학습비_전월', '이용금액_D페이_R3M', '이용건수_B페이_R3M', '이용건수_D페이_R3M', '이용금액_B페이_B0M', '이용금액_D페이_B0M', '이용건수_D페이_B0M', '이용건수_선결제_B0M', '건수_할부전환_R6M', '건수_할부전환_R3M', '금액_할부전환_R6M', '금액_할부전환_R3M', '신청건수_ATM_CA_B0', '신청건수_ATM_CL_B0', '승인거절건수_B0M', '승인거절건수_한도초과_B0M', '승인거절건수_BL_B0M', '승인거절건수_BL_R3M', '승인거절건수_입력오류_R3M', 'Segment']


In [9]:
def show_segment_distribution(df, columns, segment_col='Segment'):
    """
    각 컬럼의 고유값별로 Segment 분포 (카운트와 비율) 출력
    """
    for col in columns:
        print(f"\n\n🟢 [컬럼: {col}]")
        
        # 그룹별 개수
        count_table = df.groupby([col, segment_col]).size().unstack(fill_value=0)
        print("✅ Count:")
        display(count_table)
        
        # 그룹별 비율 (row-wise 비율)
        ratio_table = count_table.div(count_table.sum(axis=1), axis=0)
        print("✅ Ratio:")
        display(ratio_table)

In [10]:
show_segment_distribution(selected_df, cols_to_drop)



🟢 [컬럼: 이용건수_카드론_B0M]
✅ Count:


Segment,A,B,C,D,E
이용건수_카드론_B0M,,,,,
0,972,144,127587,349233,1921966
1,0,0,3,9,86


✅ Ratio:


Segment,A,B,C,D,E
이용건수_카드론_B0M,,,,,
0,0.000405,0.00006,0.053163,0.145520,0.800852
1,0.000000,0.00000,0.030612,0.091837,0.877551




🟢 [컬럼: 이용금액_카드론_B0M]
✅ Count:


...

Segment,A,B,C,D,E
승인거절건수_입력오류_R3M,,,,,
0,0.000405,0.000059,0.05316,0.145517,0.800858
1,0.000000,0.200000,0.60000,0.200000,0.000000




🟢 [컬럼: Segment]
✅ Count:


Segment,A,B,C,D,E
Segment,,,,,
A,972,0,0,0,0
B,0,144,0,0,0
C,0,0,127590,0,0
D,0,0,0,349242,0
E,0,0,0,0,1922052


✅ Ratio:


Segment,A,B,C,D,E
Segment,,,,,
A,1.0,0.0,0.0,0.0,0.0
B,0.0,1.0,0.0,0.0,0.0
C,0.0,0.0,1.0,0.0,0.0
D,0.0,0.0,0.0,1.0,0.0
E,0.0,0.0,0.0,0.0,1.0


In [11]:
import pandas as pd

def evaluate_segment_usefulness(df, columns, segment_col='Segment', rare_thresh=100):
    results = []

    for col in columns:
        value_counts = df[col].value_counts()
        rare_values = value_counts[value_counts < rare_thresh].index

        # Segment count & ratio
        count_table = df.groupby([col, segment_col]).size().unstack(fill_value=0)
        ratio_table = count_table.div(count_table.sum(axis=1), axis=0)

        # Segment별 최대-최소 비율 차이
        max_gap_per_segment = ratio_table.max() - ratio_table.min()
        total_max_gap = max_gap_per_segment.max()

        # Segment 쏠림도 (값별 최고 비율의 평균)
        max_ratios = ratio_table.max(axis=1)
        mean_peak_ratio = max_ratios.mean()

        # 희귀값 비율
        rare_ratio = df[col].isin(rare_values).mean()

        # 유의미도 판정
        if total_max_gap >= 0.3 and mean_peak_ratio >= 0.7:
            judgment = '높음'
        elif total_max_gap >= 0.1 and mean_peak_ratio >= 0.5:
            judgment = '중간'
        else:
            judgment = '낮음'

        results.append({
            '컬럼': col,
            '값 수': df[col].nunique(),
            '최대비율차(%)': round(total_max_gap * 100, 1),
            '쏠림도평균(%)': round(mean_peak_ratio * 100, 1),
            '희귀값비중(%)': round(rare_ratio * 100, 2),
            '유의미성': judgment
        })

    result_df = pd.DataFrame(results).sort_values(by='유의미성', ascending=False)
    return result_df

In [12]:
cols_to_check = cols_to_drop
result = evaluate_segment_usefulness(selected_df, cols_to_check)
display(result)

,컬럼,값 수,최대비율차(%),쏠림도평균(%),희귀값비중(%),유의미성
18,할부건수_12M_R12M,8,100.0,68.5,0.00,중간
27,RP건수_가스_B0M,2,14.8,72.8,0.00,중간
44,이용건수_D페이_B0M,2,31.8,64.3,0.00,중간
22,할부건수_무이자_14M_R12M,2,19.9,90.0,0.00,중간
43,이용금액_D페이_B0M,194,100.0,62.1,0.12,중간
42,이용금액_B페이_B0M,367,100.0,57.9,0.25,중간
41,이용건수_D페이_R3M,4,33.7,57.1,0.00,중간
40,이용건수_B페이_R3M,4,34.5,60.0,0.00,중간
28,RP건수_전기_B0M,2,19.7,70.3,0.00,중간
46,건수_할부전환_R6M,9,80.4,56.2,0.00,중간
